In [1]:
from omegaconf import OmegaConf
import os, sys
import torch
import glob
import pickle
import pandas as pd
import numpy as np
from scipy.stats import norm as scipy_norm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pytorch_lightning import Trainer
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from models import UNet, CuboidTransformerModel
from utils.visualization import report_SEVIR_statistics
from models.lightning import SEVIRPLModule

In [2]:
model_name="mearthformer_drop_0.1"
save_dir = "../models/experiments/{}/".format(model_name)
checkpoints = os.path.join(save_dir, "checkpoints/")

oc_file = os.path.join(save_dir, "cfg.yaml")
oc_from_file = OmegaConf.load(open(oc_file, "r"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset_oc = OmegaConf.to_object(oc_from_file.dataset)
micro_batch_size=1
dm = SEVIRPLModule.get_sevir_datamodule(
        dataset_oc=dataset_oc,
        micro_batch_size=micro_batch_size,
        num_workers=8,)
dm.prepare_data()
dm.setup()

In [6]:
ckpts = glob.glob(checkpoints + '*.ckpt')
ckpts.sort()
curr_ckpt = ckpts[1]
print(ckpts)
print(curr_ckpt)

['../models/experiments/earthformer/checkpoints/model-epoch=039.ckpt', '../models/experiments/earthformer/checkpoints/last.ckpt', '../models/experiments/earthformer/checkpoints/model-epoch=003.ckpt', '../models/experiments/earthformer/checkpoints/last-v1.ckpt']
../models/experiments/earthformer/checkpoints/last.ckpt


In [7]:
total_num_steps = SEVIRPLModule.get_total_num_steps(
    epoch=oc_from_file.optim.max_epochs,
    num_samples=dm.num_train_samples,
    total_batch_size=oc_from_file.optim.total_batch_size,
)
pl_module = SEVIRPLModule.load_from_checkpoint(curr_ckpt, total_num_steps=total_num_steps, save_dir=model_name, oc_file=oc_file)
accumulate_grad_batches = oc_from_file.optim.total_batch_size // (oc_from_file.optim.micro_batch_size * 1)
trainer_kwargs = pl_module.set_trainer_kwargs(
    devices= 1,
    accumulate_grad_batches=accumulate_grad_batches,
)
trainer_kwargs['strategy'] = 'auto'
if len(trainer_kwargs['logger']) == 3:
    trainer_kwargs['logger'].pop()
trainer = Trainer(**trainer_kwargs)

/home/idies/workspace/Storage/akhot2/persistent/earthformer/lib/python3.12/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.test(model=pl_module, datamodule=dm)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 56/12159 [00:07<25:20,  7.96it/s] 

/home/idies/workspace/Storage/akhot2/persistent/earthformer/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
